In [2]:
%pip install sentencepiece pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\tanma\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
import sentencepiece as spm
import pandas as pd
train_df = pd.read_csv("data\eng.sentence.train.tsv", sep="\t", header=None)
training_txt = train_df[0].values.tolist()  + train_df[1].values.tolist()
training_txt = [str(sent) for sent in training_txt]
with open('data\\training_bpe.txt', 'w', encoding='utf-8') as file:
  file.writelines(f"{line}\n" for line in training_txt)
spm.SentencePieceTrainer.train('--input=data\\training_bpe.txt --model_prefix=models\m_bpe --vocab_size=1800 --model_type=bpe')
sp_bpe = spm.SentencePieceProcessor()
sp_bpe.load('models\m_bpe.model')

True

In [4]:
test_df = pd.read_csv("data\eng.sentence.test.tsv", sep="\t", header=None)
encoded = []
test_list = test_df[0].values.tolist()
i = 0
for sentence in test_list:
  enc = sp_bpe.encode_as_pieces(sentence)
  encoded.append(enc)
  print(enc)
  i += 1
  if i > 100:
    break

['▁What', '▁if', '▁G', 'oo', 'g', 'le', '▁M', 'or', 'p', 'hed', '▁I', 'nt', 'o', '▁G', 'oo', 'g', 'le', 'O', 'S', '▁?']
['▁What', '▁if', '▁G', 'oo', 'g', 'le', '▁exp', 'and', 'ed', '▁on', '▁its', '▁se', 'arch', '▁-', '▁en', 'g', 'ine', '▁(', '▁and', '▁now', '▁e', '-', 'mail', '▁)', '▁w', 'are', 's', '▁into', '▁a', '▁full', '▁-', '▁fl', 'ed', 'ged', '▁oper', 'ating', '▁system', '▁?']
['▁', '[', '▁v', 'ia', '▁M', 'ic', 'ro', 'so', 'ft', '▁W', 'atch', '▁from', '▁M', 'ary', '▁J', 'o', '▁F', 'o', 'ley', '▁', ']']
['▁(', '▁And', '▁,', '▁by', '▁the', '▁way', '▁,', '▁is', '▁any', 'b', 'ody', '▁else', '▁just', '▁a', '▁little', '▁n', 'ost', 'al', 'g', 'ic', '▁for', '▁the', '▁days', '▁when', '▁that', '▁was', '▁a', '▁good', '▁thing', '▁?', '▁)']
['▁This', '▁B', 'u', 'z', 'z', 'M', 'ach', 'ine', '▁post', '▁ar', 'g', 'ues', '▁that', '▁G', 'oo', 'g', 'le', "▁'", 's', '▁r', 'ush', '▁to', 'ward', '▁u', 'b', 'i', 'qu', 'ity', '▁might', '▁back', 'f', 'ire', '▁--', '▁which', '▁we', "▁'", 've', '▁all', '▁h

In [ ]:
df_guess = pd.read_csv('data\eng.sentence.test.gold.tsv', sep='\t', header=None)
i = 0
for entry in test_list:
  sent = ''
  list_sent = sp_bpe.encode_as_pieces(entry)
  list_sent = [w.replace("_", "") for w in list_sent]
  j = 0
  length = len(list_sent)
  for word in list_sent:
    if j != length - 1:
      sent += (word + ' @@')
      j+=1
    else :
      sent += word
  df_guess[1][i] = sent
  i+=1

df_guess.to_csv("outputs\eng.sentence.test.bpe_guess.tsv", sep='\t', header=None, index = False)